In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from plots_file import *
from our_functionsv3 import *

np.random.seed(0)
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.callbacks import EarlyStopping
import time
import matplotlib
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
from LSTM_functions import *

### Load data

In [4]:
X_train, Y_train = get_data(pos = "twitter-datasets/train_pos_preprocessed_to_use.txt", neg = "twitter-datasets/train_neg_preprocessed_to_use.txt")
Y_train_oh = convert_to_one_hot(Y_train, C=2)

In [5]:
ids, _ = get_test_data("twitter-datasets/test_data.txt")
X_test = read_data("twitter-datasets/test_preprocessed_to_use.txt")

In [6]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

### Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences

In [7]:
max_length = get_max_length(X_train)

In [8]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_length)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_length)

199999 words were not in the dictionary
9999 words were not in the dictionary


### Submission

In [9]:
histories = []
models = []

optimizer = ['adam']
for opt in optimizer:
    history, model = complete_model(X_train_indices, Y_train_oh, word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                                     epochs = 50, loss ='binary_crossentropy', optimizer = opt)
    histories.append(history)
    models.append(model)
    label = predict_lstm(model, X_test_indices)
    path = 'submissions/submission_model_200_emb_dim.csv'
    create_csv_submission(ids, label, path)

Train on 180000 samples, validate on 20000 samples
Epoch 1/50
  1344/180000 [..............................] - ETA: 16:58 - loss: 0.6891 - acc: 0.5223

KeyboardInterrupt: 